In [1]:
import tensorflow as tf
import keras
from skimage.transform import resize

from keras.regularizers import l2
from keras.models import Model
from keras.applications import mobilenet_v2, mobilenet, resnet50, densenet
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, \
    BatchNormalization, Activation, GlobalAveragePooling2D, DepthwiseConv2D, \
    Dropout, ReLU, Concatenate, Input, add

from keras.callbacks import ModelCheckpoint
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.callbacks import CSVLogger
from datetime import datetime

import os.path
from keras.utils.vis_utils import plot_model
from scipy.spatial import distance
import scipy.io as sio




def sample_res_net_v0(self,input_shape, output_shape):

    input = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))

    #block_1
    b1_cnv2d_1 = Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), padding='same',
                     use_bias=False, name='b1_cnv2d_1', kernel_initializer='normal')(input)
    b1_relu_1 = ReLU(name='b1_relu_1')(b1_cnv2d_1)
    b1_bn_1 = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b1_bn_1')(b1_relu_1)  # size: 14*14

    b1_cnv2d_2 = Conv2D(filters=32, kernel_size=(1, 1), strides=(2, 2), padding='same',
                        use_bias=False, name='b1_cnv2d_2', kernel_initializer='normal')(b1_bn_1)
    b1_relu_2 = ReLU(name='b1_relu_2')(b1_cnv2d_2)
    b1_out = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b1_out')(b1_relu_2)  # size: 14*14

    #block 2
    b2_cnv2d_1 = Conv2D(filters=32, kernel_size=(1, 1), strides=(1, 1), padding='same',
                        use_bias=False, name='b2_cnv2d_1', kernel_initializer='normal')(b1_out)
    b2_relu_1 = ReLU(name='b2_relu_1')(b2_cnv2d_1)
    b2_bn_1 = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b2_bn_1')(b2_relu_1)  # size: 14*14

    b2_add = add([b1_out, b2_bn_1])  #

    b2_cnv2d_2 = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same',
                        use_bias=False, name='b2_cnv2d_2', kernel_initializer='normal')(b2_add)
    b2_relu_2 = ReLU(name='b2_relu_2')(b2_cnv2d_2)
    b2_out = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b2_bn_2')(b2_relu_2)  # size: 7*7

    #block 3
    b3_cnv2d_1 = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same',
                        use_bias=False, name='b3_cnv2d_1', kernel_initializer='normal')(b2_out)
    b3_relu_1 = ReLU(name='b3_relu_1')(b3_cnv2d_1)
    b3_bn_1 = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b3_bn_1')(b3_relu_1)  # size: 7*7

    b3_add = add([b2_out, b3_bn_1])  #

    b3_cnv2d_2 = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same',
                        use_bias=False, name='b3_cnv2d_2', kernel_initializer='normal')(b3_add)
    b3_relu_2 = ReLU(name='b3_relu_2')(b3_cnv2d_2)
    b3_out = BatchNormalization(epsilon=1e-3, momentum=0.999, name='b3_out')(b3_relu_2)  # size: 3*3

    #block 4
    b4_avg_p = GlobalAveragePooling2D()(b3_out)
    output = Dense(output_shape, name='model_output', activation='softmax',
                   kernel_initializer='he_uniform')(b4_avg_p)

    model = Model(input, output)

    model_json = model.to_json()

    with open("sample_res_net_v0.json", "w") as json_file:
        json_file.write(model_json)
    model.summary()
    return model


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000, 1000,  0                                            
__________________________________________________________________________________________________
b1_cnv2d_1 (Conv2D)             (None, 500, 500, 16) 144         input_1[0][0]                    
__________________________________________________________________________________________________
b1_relu_1 (ReLU)                (None, 500, 500, 16) 0           b1_cnv2d_1[0][0]                 
__________________________________________________________________________________________________
b1_bn_1 (BatchNormalization)    (None, 500, 500, 16) 64          b1_relu_1[0][0]                  
______________________________________________________________________________________________

In [13]:
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.utils import shuffle
import os
from sklearn.model_selection import train_test_split
from numpy import save, load, asarray
import os.path
from keras import losses
from keras import backend as K
Brains = []
Brain_Types = []
Brain_Data = []
listofLMCI = []
listofEMCI = []
listofCN = []

listofLMCI = os.listdir("./LMCI")
listofEMCI = os.listdir("./EMCI")
listofCN = os.listdir("./CN")
CN = 0
EMCI = 1
LMCI = 2




def train_model(input_shape,output_shape):
    for i in range(len(listofCN)):
        print(i)
        Brain_Data = []
        #cn
        file = os.path.join("./CN",listofCN[i])
        with open(file) as f:
            lines = f.readlines()
            for line in lines:
                x = float(line.strip())
                Brain_Data.append(x)
        Brains.append(Brain_Data)
        Brain_Types.append(CN)

        #emci
        Brain_Data = []
        file = os.path.join("./EMCI",listofEMCI[i])
        with open(file) as f:
            lines = f.readlines()
            for line in lines:
                x = float(line.strip())
                Brain_Data.append(x)
        Brains.append(Brain_Data)
        Brain_Types.append(EMCI)
        #lmci
        Brain_Data = []
        file = os.path.join("./LMCI",listofLMCI[i])
        with open(file) as f:
            lines = f.readlines()
            for line in lines:
                x = float(line.strip())
                Brain_Data.append(x)
        Brains.append(Brain_Data)
        Brain_Types.append(LMCI)
    

    x_train, x_test, y_train, y_test = train_test_split(Brains, Brain_Types, test_size=0.16)
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    '''take 5,000 samples from train to be used as validation set'''
    x_val = x_train[-1:]
    y_val = y_train[-1:]
    x_train = x_train[:-1]
    y_train = y_train[:-1]

    '''adopting the images shape, from ?*28*28 to ?*28*28*1'''
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    '''normalizing images'''
    x_train = x_train/255.0
    x_val = x_val/255.0
    x_test = x_test/255.0

    '''create categorical labels'''
    y_train = keras.utils.to_categorical(y_train, output_shape)
    y_val = keras.utils.to_categorical(y_val, output_shape)
    y_test = keras.utils.to_categorical(y_test, output_shape)

    '''creating model:'''
    model_1 = sample_res_net_v0(input_shape=input_shape, output_shape=output_shape)

    '''preparing network for being trained'''
    model_1.compile(loss=losses.categorical_crossentropy,
                    optimizer='adam',
                    metrics=['accuracy'])

    '''start training the model'''
    history = model_1.fit(x_train, y_train,
                          batch_size=100,
                          epochs=2,
                          validation_data=(x_val, y_val))
    model_1.save("res_model.hd5")

    self.show_figures(history.history)


def show_figures(self, history):
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
    plt.savefig('accuracy')

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
    plt.savefig('loss')

print("Start")
train_model([1000, 1000, 1],3)


Start
0
1
2


IndexError: index 5 is out of bounds for axis 1 with size 3